In [7]:
import numpy as np
from scipy.stats import norm

In [8]:
total_products = 13000
N1 = 8000
mu1 = 13
sigma1 = 0.5
N2 = 5000
mu2 = 12
sigma2 = 0.7
p1 = 0.85
p2 = 0.8

In [9]:
mu1_hat = np.random.rand()
mu2_hat = np.random.rand()
sigma1_hat = np.random.rand()
sigma2_hat = np.random.rand()

In [15]:
for _ in range(100): 
    w1 = (p1 * norm.pdf(mu1, mu1_hat, sigma1_hat)) / (
        p1 * norm.pdf(mu1, mu1_hat, sigma1_hat) + p2 * norm.pdf(mu1, mu2_hat, sigma2_hat)
    )
    w2 = 1 - w1
    
    epsilon = 1e-10
    if np.abs(np.sum(w1)) < epsilon:
        w1 = epsilon
    if np.abs(np.sum(w2)) < epsilon:
        w2 = epsilon
    
    mu1_hat = np.sum(w1 * mu1) / np.sum(w1)
    mu2_hat = np.sum(w2 * mu2) / np.sum(w2)
    
    min_sigma = 0.1
    sigma1_hat = max(sigma1_hat, min_sigma)
    sigma2_hat = max(sigma2_hat, min_sigma)
    sigma1_hat = np.sqrt(np.sum(w1 * (mu1 - mu1_hat) ** 2) / np.sum(w1))
    sigma2_hat = np.sqrt(np.sum(w2 * (mu2 - mu2_hat) ** 2) / np.sum(w2))



In [14]:
proba_X = []
for i in range(total_products):
    p_x1 = w1 * norm.pdf(mu1, mu1_hat, sigma1_hat)
    p_x2 = w2 * norm.pdf(mu2, mu2_hat, sigma2_hat)
    proba_temp = [p_x1 / (p_x1 + p_x2), p_x2 / (p_x1 + p_x2)]
    proba_X.append(proba_temp)

proba_X = np.array(proba_X)
predicted_labels = np.argmax(proba_X, axis=1) + 1  

y = np.concatenate((np.ones(N1), 2 * np.ones(N2)))

print("Доля вірно визначених виробів:", np.sum(predicted_labels == y) / total_products)

Доля вірно визначених виробів: 0.6153846153846154


In [20]:
import numpy as np

total_products = 13000
factory1_products = 8000
factory2_products = 5000

factory1_mean_weight = 13
factory1_weight_std = 0.5

factory2_mean_weight = 12
factory2_weight_std = 0.7

factory1_quality_proportion = 0.85
factory2_quality_proportion = 0.8

estimated_factory1_proportion = 0.5
estimated_factory2_proportion = 0.5

num_iterations = 10

for _ in range(num_iterations):
    factory1_weights = (
        factory1_quality_proportion
        * np.random.normal(factory1_mean_weight, factory1_weight_std, total_products)
    )
    factory2_weights = (
        factory2_quality_proportion
        * np.random.normal(factory2_mean_weight, factory2_weight_std, total_products)
    )

    likelihood_factory1 = (
        estimated_factory1_proportion
        * np.exp(-0.5 * ((factory1_weights - factory1_mean_weight) / factory1_weight_std) ** 2)
    )
    likelihood_factory2 = (
        estimated_factory2_proportion
        * np.exp(-0.5 * ((factory2_weights - factory2_mean_weight) / factory2_weight_std) ** 2)
    )

    total_likelihood = likelihood_factory1 + likelihood_factory2

    factory1_posterior = likelihood_factory1 / total_likelihood
    factory2_posterior = likelihood_factory2 / total_likelihood

    estimated_factory1_proportion = np.mean(factory1_posterior)
    estimated_factory2_proportion = np.mean(factory2_posterior)

print("Estimated probabilities:")
print("Factory 1:", estimated_factory1_proportion)
print("Factory 2:", estimated_factory2_proportion)

if estimated_factory1_proportion > estimated_factory2_proportion:
    print("The product was likely manufactured in Factory 1.")
else:
    print("The product was likely manufactured in Factory 2.")

Estimated probabilities:
Factory 1: 0.28243759184844963
Factory 2: 0.7175624081515503
The product was likely manufactured in Factory 2.
